In [ ]:
# Con la página dinámica
import numpy as np
import pandas as pd
import os
import glob
import pytest
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

os.chdir(r"C:\Users\Andrés\OneDrive\Desktop\jupyter notebook")
options = webdriver.ChromeOptions()

# Inicia con la ventana maximizada
options.add_argument("--start-maximized")
driver = webdriver.Chrome(r"C:\Users\IDEAPAD530S\anaconda3\Lib\site-packages\chromedriver_binary\chromedriver.exe", chrome_options=options)

pags = 11

for clicker in np.arange(0,pags,1):
    # Abre el URL
    driver.get("http://www.congreso.gob.pe/pley-2016-2021")

    # Trabajemos con iFrames
    driver.switch_to.frame(0)
    clicker_h = clicker
    
    if clicker!=0:
        # Da click para el siguiente tramo de PL
        while clicker_h > 0:
            driver.find_element(By.XPATH, "//img[contains(@src, '/Sicr/TraDocEstProc/CLProLey2016.nsf/8eac1ef603908b5105256cdf006c41b1/$Body/0.D90?OpenElement&FieldElemFormat=gif')]").click()
            time.sleep(2)
            clicker_h = clicker_h - 1 
    else:
        pass
    element = driver.find_element_by_xpath('//*')
    element = element.get_attribute('innerHTML')
    sub_url = 'http://www2.congreso.gob.pe/'

    # VARIABLES DE LA RUTA ORIGINAL
    a = driver.find_elements(By.XPATH,'//a[contains(@href, "/Sicr/TraDocEstProc/CLProLey2016.nsf/641842f7e5d631bd052578e20058a231")]')
    d = driver.find_elements(By.XPATH,'//td/font[contains(@size, "1") and (contains(@face, "Verdana"))]')
    b = driver.find_elements(By.XPATH,'//td[contains(@align, "center")][3]/font[contains(@size, "2") and (contains(@face, "Verdana"))]')
    c = driver.find_elements(By.XPATH,'//td[contains(@align, "center")][4]/font[contains(@size, "2") and (contains(@face, "Verdana"))]')
    links = driver.find_elements(By.XPATH,'//td[contains(@align, "center")][1]/font[contains(@size, "2") and (contains(@face, "Verdana"))]/a')
    
    # VARIABLES RECURSIVAS (POR CADA URL) EXCEPTUANDO LA PRIMERA
    links_t = []
    pdfs = []
    partidos = []
    sumillas = []
    autores = []

    for i in np.arange(0,len(links),1):
        #print(links[i].get_attribute("href"))
        links_t2 = links[i].get_attribute("href")
        links_t.append(links_t2)

    # VARIABLES ORIGINALES
    x = []
    y = []
    z = []
    w = []

    for i in np.arange(0,len(a),1):
        print(a[i].text, b[i].text, c[i].text, d[i].text)
        x.append(a[i].text)
        y.append(b[i].text)
        z.append(c[i].text)
        w.append(d[i].text)

    # ENTRANDO A CADA URL Y EXTRAYENDO PDF, PARTIDO Y SUMILLA    
    for i in np.arange(0,len(links_t),1):
        url = links_t[i]
        driver.get(url)
        window_before = driver.window_handles[0]
        partido = driver.find_element(By.XPATH, '//tr[contains(@valign,"top")][6]/td[contains(@width,"472")]/font')
        partidos.append(partido.text)
        sumilla = driver.find_element(By.XPATH, '//tr[contains(@valign,"top")][8]/td[contains(@width,"472")]/font')
        sumillas.append(sumilla.text)
        autor = driver.find_element(By.XPATH, '//tr[contains(@valign,"top")][9]/td[contains(@width,"472")]/font')
        autores.append(autor.text)
        driver.find_element(By.LINK_TEXT, "Ver Expediente Digital").click()
        window_after = driver.window_handles[1]
        driver.switch_to_window(window_after)
        pdf = driver.find_element(By.XPATH, '//td[contains(@width,"61%") and (contains(@bgcolor, "#FFFFFF"))]//a')
        pdfs.append(pdf.get_attribute("href"))
        driver.close()
        driver.switch_to_window(window_before)

    data = list(zip(x,y,z,w,links_t,partidos,sumillas,pdfs,autores))
    columns = ['Número','Fecha Presentación','Estado','Título del Proyecto','URL','Partido','Sumilla','PDF','Autores']
    data = pd.DataFrame(data, columns=columns)
    excel = 'congreso' + str(clicker) + '.xlsx'
    data.to_excel(excel,index=False)
    
df = 'congreso' + str(0) + '.xlsx'
df = pd.read_excel(df)

for i in np.arange(1,pags,1):
    data = 'congreso' + str(i) + '.xlsx'
    datas = pd.read_excel(data)
    df = df.append(datas, ignore_index = True)

# BASE DE DATOS FINAL
df = df.drop_duplicates()
df.to_excel('compilado_congreso.xlsx', index=False)

# REMOVIENDO ARCHIVOS INNECESARIOS
for f in glob.glob("congreso*.xlsx"):
    os.remove(f)